<a href="https://colab.research.google.com/github/EduardoAve/Data-science-portfolio/blob/main/notebooks/variability_imputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [3]:
import pandas as pd
import numpy as np

# --- URLs de los archivos CSV en GitHub ---
url_vulnerability = 'https://raw.githubusercontent.com/EduardoAve/Labour-well-being/refs/heads/main/data/01_raw-00/vulnerability.csv'
url_dataset = 'https://raw.githubusercontent.com/EduardoAve/Labour-well-being/refs/heads/main/data/01_raw-00/Final_Dataset.csv'

# --- Cargar los datos en los DataFrames ---

# df1 cargará los datos de vulnerability.csv
print("Cargando datos de vulnerability...")
df1 = pd.read_csv(url_vulnerability)

# df2 cargará los datos de Final_Dataset.csv
print("Cargando datos de Final_Dataset...")
df2 = pd.read_csv(url_dataset)

Cargando datos de vulnerability...
Cargando datos de Final_Dataset...


In [4]:
import pandas as pd
# numpy es necesario para el redondeo
import numpy as np
# Importar KNNImputer para la imputación
try:
    from sklearn.impute import KNNImputer
    sklearn_disponible = True
except ImportError:
    sklearn_disponible = False
    print("Advertencia: La librería 'scikit-learn' no está instalada.")
    print("No se realizará la imputación KNN. Para habilitarla, ejecuta: pip install scikit-learn")


# --- Asumimos que df1 y df2 ya están cargados ---
# Por ejemplo:
# df1 = pd.read_csv('ruta/a/tu/df1.csv')
# df2 = pd.read_csv('ruta/a/tu/df2.csv')
# O desde Excel:
# df1 = pd.read_excel('ruta/a/tu/archivo.xlsx', sheet_name='Hoja_df1')
# df2 = pd.read_excel('ruta/a/tu/archivo.xlsx', sheet_name='Hoja_df2')

# --- Verifica las formas iniciales (opcional pero recomendado) ---
print("--- Formas Iniciales de los DataFrames Cargados ---")
try:
    print(f"Forma df1: {df1.shape}")
    print(f"Forma df2: {df2.shape}")
    # Validar número de columnas esperado en df1
    if df1.shape[1] != 45:
        print("Advertencia: df1 no tiene las 45 columnas esperadas.")
    # Validar que df2 tenga al menos 45 columnas
    if df2.shape[1] < 45:
         print("Error: df2 tiene menos de 45 columnas, no se puede procesar.")
         # Podrías detener la ejecución aquí si es un error crítico
         exit()
    # Validar que tengan el mismo número de filas
    if df1.shape[0] != df2.shape[0]:
        print("Advertencia: df1 y df2 no tienen el mismo número de filas. El filtro podría no funcionar como se espera si los índices no coinciden perfectamente.")

except NameError:
    print("Error: Asegúrate de que las variables 'df1' y 'df2' existan y contengan tus DataFrames.")
    # Detener ejecución si los dataframes no existen
    exit()
except Exception as e:
    print(f"Ocurrió un error al verificar los DataFrames: {e}")
    # Detener ejecución si hay problemas
    exit()
print("-" * 30)


# --- Definir constantes ---
num_preguntas_vulnerabilidad = 44
# Criterio: Eliminar si hay MÁS de 4 nulos.
# Por lo tanto, mantenemos si hay 4 o MENOS nulos.
max_nulos_permitidos = 4

# --- Identificar las columnas de preguntas en df2 ---
# Son las 'num_preguntas_vulnerabilidad' columnas ANTES de la última columna de df2
try:
    total_cols_df2 = df2.shape[1]
    inicio_preguntas_df2 = total_cols_df2 - 1 - num_preguntas_vulnerabilidad
    fin_preguntas_df2 = total_cols_df2 - 1 # El índice final para iloc es exclusivo
    columnas_preguntas_df2 = df2.iloc[:, inicio_preguntas_df2:fin_preguntas_df2]

    # --- Calcular cuántas preguntas SÍ fueron contestadas (NaNs) por registro en df2 ---
    # Contar valores NULOS en las columnas de preguntas de df2
    nulos_por_registro = columnas_preguntas_df2.isnull().sum(axis=1)

    # --- Crear el filtro: Mantener registros con 4 o MENOS valores nulos ---
    # El criterio es eliminar si nulos > 4, por lo tanto, mantenemos si nulos <= 4
    filtro_mantener = nulos_por_registro <= max_nulos_permitidos

    print(f"\nSe aplicará filtro para MANTENER registros con {max_nulos_permitidos} o menos NULOS (preguntas no contestadas).")
    print(f"Número de registros a mantener: {filtro_mantener.sum()}")
    print(f"Número de registros a eliminar (con más de {max_nulos_permitidos} nulos): {len(df1) - filtro_mantener.sum()}")
    print("-" * 30)

    # --- Aplicar el filtro a ambos DataFrames ---
    # Se asume que los índices de df1 y df2 están alineados
    df1_depurado = df1[filtro_mantener].copy()
    df2_depurado = df2[filtro_mantener].copy()

    print("--- DataFrames Depurados (Antes de Imputación) ---")
    print(f"Forma df1 depurado: {df1_depurado.shape}")
    print(f"Forma df2 depurado: {df2_depurado.shape}")
    print("-" * 30)

    # --- Imputación KNN (si sklearn está disponible) ---
    if sklearn_disponible and not df1_depurado.empty and not df2_depurado.empty:
        print("--- Realizando Imputación KNN ---")
        # Configurar el imputador KNN (puedes ajustar n_neighbors)
        imputer = KNNImputer(n_neighbors=5)

        # --- Imputar df1_depurado ---
        print("Imputando df1...")
        # Seleccionar las columnas de preguntas (primeras num_preguntas_vulnerabilidad)
        cols_preguntas_df1_idx = df1_depurado.columns[:num_preguntas_vulnerabilidad]
        cols_a_imputar_df1 = df1_depurado[cols_preguntas_df1_idx]
        col_vulnerabilidad_df1 = df1_depurado.iloc[:, -1] # Última columna

        # Aplicar imputación
        imputed_data_df1 = imputer.fit_transform(cols_a_imputar_df1)
        # *** NUEVO: Redondear y convertir a entero ***
        imputed_data_df1 = np.round(imputed_data_df1).astype(np.int64)
        df_imputed_part_df1 = pd.DataFrame(imputed_data_df1, columns=cols_preguntas_df1_idx, index=df1_depurado.index)

        # Reconstruir df1_depurado con datos imputados
        df1_depurado = pd.concat([df_imputed_part_df1, col_vulnerabilidad_df1], axis=1)
        print("df1 imputado y redondeado.")

        # --- Imputar df2_depurado ---
        print("Imputando df2...")
        # Identificar columnas en df2_depurado (posiciones relativas pueden cambiar si df2 original tenía más de 45 cols)
        total_cols_df2_depurado = df2_depurado.shape[1]
        inicio_preguntas_df2_depurado = total_cols_df2_depurado - 1 - num_preguntas_vulnerabilidad
        fin_preguntas_df2_depurado = total_cols_df2_depurado - 1

        # Seleccionar partes de df2_depurado
        cols_antes_df2 = df2_depurado.iloc[:, :inicio_preguntas_df2_depurado] # Columnas antes de las preguntas
        cols_preguntas_df2_idx = df2_depurado.columns[inicio_preguntas_df2_depurado:fin_preguntas_df2_depurado]
        cols_a_imputar_df2 = df2_depurado[cols_preguntas_df2_idx]
        col_vulnerabilidad_df2 = df2_depurado.iloc[:, -1] # Última columna

        # Aplicar imputación
        # Nota: Usamos el mismo imputer ajustado en df1 o ajustamos uno nuevo.
        imputed_data_df2 = imputer.fit_transform(cols_a_imputar_df2)
        # *** NUEVO: Redondear y convertir a entero ***
        imputed_data_df2 = np.round(imputed_data_df2).astype(np.int64)
        df_imputed_part_df2 = pd.DataFrame(imputed_data_df2, columns=cols_preguntas_df2_idx, index=df2_depurado.index)

        # Reconstruir df2_depurado con datos imputados
        partes_df2 = []
        if not cols_antes_df2.empty:
            partes_df2.append(cols_antes_df2)
        partes_df2.append(df_imputed_part_df2)
        partes_df2.append(col_vulnerabilidad_df2)
        df2_depurado = pd.concat(partes_df2, axis=1)
        print("df2 imputado y redondeado.")
        print("-" * 30)

        print("--- DataFrames Depurados e Imputados (Enteros) ---")
        print(f"Forma df1 depurado e imputado: {df1_depurado.shape}")
        print(f"Forma df2 depurado e imputado: {df2_depurado.shape}")
        # Verificar si quedan NaNs en las columnas imputadas (no debería haber)
        print(f"NaNs restantes en preguntas df1: {df1_depurado.iloc[:,:num_preguntas_vulnerabilidad].isnull().sum().sum()}")
        print(f"NaNs restantes en preguntas df2: {df2_depurado.iloc[:, inicio_preguntas_df2_depurado:fin_preguntas_df2_depurado].isnull().sum().sum()}")
        # Verificar tipo de dato (debería ser entero)
        print(f"Tipo de dato ejemplo en preguntas df1: {df1_depurado.iloc[0, 0].dtype}")
        print(f"Tipo de dato ejemplo en preguntas df2: {df2_depurado.iloc[0, inicio_preguntas_df2_depurado].dtype}")
        print("-" * 30)

    elif not sklearn_disponible:
         print("Imputación KNN omitida porque 'scikit-learn' no está instalado.")
    elif df1_depurado.empty or df2_depurado.empty:
         print("Imputación KNN omitida porque uno o ambos DataFrames depurados están vacíos.")


    # --- Guardar los DataFrames (depurados y posiblemente imputados) en un mismo archivo Excel ---
    nombre_archivo_excel = 'dataset_vulnerability_new.xlsx' # Nombre de archivo

    try:
        with pd.ExcelWriter(nombre_archivo_excel, engine='openpyxl') as writer:
            # Hoja 1: df2 con nombre 'dataset'
            df2_depurado.to_excel(writer, sheet_name='dataset', index=False)
            # Hoja 2: df1 con nombre 'vulnerability'
            df1_depurado.to_excel(writer, sheet_name='vulnerability', index=False)

        print(f"Archivos depurados e imputados (enteros) guardados correctamente en '{nombre_archivo_excel}'")
        print(f"  - Hoja 'dataset': Contiene df2 depurado e imputado.")
        print(f"  - Hoja 'vulnerability': Contiene df1 depurado e imputado.")

    except ImportError:
        print("\nError: Necesitas instalar la librería 'openpyxl' para escribir archivos .xlsx.")
        print("Ejecuta: pip install openpyxl")
    except Exception as e:
        print(f"\nOcurrió un error al guardar el archivo Excel: {e}")

except IndexError:
     print(f"Error: Problema al acceder a las columnas de df2. Verifica que df2 tenga al menos {num_preguntas_vulnerabilidad + 1} columnas.")
except Exception as e:
     print(f"Ocurrió un error inesperado durante el procesamiento: {e}")


--- Formas Iniciales de los DataFrames Cargados ---
Forma df1: (2747, 45)
Forma df2: (2747, 132)
------------------------------

Se aplicará filtro para MANTENER registros con 4 o menos NULOS (preguntas no contestadas).
Número de registros a mantener: 2708
Número de registros a eliminar (con más de 4 nulos): 39
------------------------------
--- DataFrames Depurados (Antes de Imputación) ---
Forma df1 depurado: (2708, 45)
Forma df2 depurado: (2708, 132)
------------------------------
--- Realizando Imputación KNN ---
Imputando df1...
df1 imputado y redondeado.
Imputando df2...
df2 imputado y redondeado.
------------------------------
--- DataFrames Depurados e Imputados (Enteros) ---
Forma df1 depurado e imputado: (2708, 45)
Forma df2 depurado e imputado: (2708, 132)
NaNs restantes en preguntas df1: 0
NaNs restantes en preguntas df2: 0
Tipo de dato ejemplo en preguntas df1: int64
Tipo de dato ejemplo en preguntas df2: int64
------------------------------
Archivos depurados e imputados 